In [1]:
#Camille Chow
#ECE 471 Assignment 2

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tqdm import tqdm

BATCH_SIZE = 40
NUM_BATCHES = 300

In [2]:
class Data():
    def __init__(self):
        num_samp = 200
        sigma = 0.1
        np.random.seed(1)           #change after testing

        #training data for label 0
        self.index = np.arange(num_samp)        
        self.t0 = np.random.uniform(.08, 1.8, num_samp)
        self.x0 = np.atleast_2d(6 * self.t0 * np.cos(2*np.pi*self.t0) + np.random.normal(0, sigma, num_samp)).T
        self.y0 = np.atleast_2d(6 * self.t0 * np.sin(2*np.pi*self.t0) + np.random.normal(0, sigma, num_samp)).T
        self.class0 = np.vstack((self.x0, self.y0))
        #training data for label 1
        self.t1 = np.random.uniform(.08, 1.8, num_samp)
        self.x1 = np.atleast_2d(6 * self.t1 * np.cos(2*np.pi*self.t1 + np.pi) + np.random.normal(0, sigma, num_samp)).T
        self.y1 = np.atleast_2d(6 * self.t1 * np.sin(2*np.pi*self.t1 + np.pi) + np.random.normal(0, sigma, num_samp)).T
        self.class1 = np.vstack((self.x1, self.y1))

        self.labels = np.hstack((np.zeros(num_samp), np.ones(num_samp)))
        self.coords = np.hstack((self.class0, self.class1))

    def get_batch(self):
        choices = np.random.choice(self.index, size=BATCH_SIZE)

        return self.coords[choices], self.labels[choices]      #flatten????


In [4]:
def f(x):   #takes x in R2 and outputs sigmoid
    
    layer1_size = 250
    layer2_size = 250                        #experiment with layer sizes
    
    w1 = tf.get_variable('w1', [2, layer1_size], tf.float32, tf.random_normal_initializer())    
    b1 = tf.get_variable('b1', [layer1_size], tf.float32, tf.zeros_initializer())
    w2 = tf.get_variable('w2', [layer1_size, layer2_size], tf.float32, tf.random_normal_initializer())    
    b2 = tf.get_variable('b2', [layer2_size], tf.float32, tf.zeros_initializer())
    w3 = tf.get_variable('w3', [layer2_size, 1], tf.float32, tf.random_normal_initializer())    
    b3 = tf.get_variable('b3', [], tf.float32, tf.zeros_initializer())                  #other initializers???

    #perceptron
    layer1 = tf.add(tf.matmul(x, w1), b1)
    layer2 = tf.add(tf.matmul(tf.nn.relu(layer1), w2), b2)
    out = tf.add(tf.matmul(tf.nn.relu(layer2), w3), b3)
                                                                #try other activation functions???
    return tf.squeeze(tf.sigmoid(out))

In [6]:
x = tf.placeholder(tf.float32, [BATCH_SIZE,2])
t = tf.placeholder(tf.float32, [BATCH_SIZE])
logits = f(x)

lam = .5                 #experiment here
loss = tf.losses.sigmoid_cross_entropy(t, logits) + tf.add_n([lam * tf.nn.l2_loss(v) for v in tf.trainable_variables()])
optim = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(loss)        #other optimizers???
init = tf.global_variables_initializer()

sess = tf.Session()
sess.run(init)

data = Data()

for _ in tqdm(range(0, NUM_BATCHES)):
    x_np, t_np = data.get_batch()
    loss_np, _ = sess.run([loss, optim], feed_dict={x: x_np, t: t_np})

100%|██████████| 300/300 [00:04<00:00, 60.45it/s]
